In [ ]:
import requests
import pandas as pd

# Credentials
consumer_key = "aBqUYvVqFhPhst8FXy4ZHa9GU"
consumer_secret = "TM9wwYNKca1j85wBh21m03ZyEKVv4aNHHv3pULnPsYXXWp4cRk"
access_token = "782220683564748800-B5pGV21xe5mfdAjeaR9eSUTF5dWDNz8"
access_token_secret = "kr2E9bCOHwfjElkGHB1UdAHqtOkIUaUWfC1GQhwZ2N5cR"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJjYewEAAAAAm6PKdlVVO6XY%2FchLn%2BIdpQ0YCig%3DjwF0CFBFwNkqbmIa4bbrcMhZhlAO1bcLRDFBIW4yYDv4ycvZj8"
headers = {"Authorization": "Bearer {}".format(bearer_token)}

## Fetch tweets related to the various sectors: health, transportation, education, and food

In [115]:
sector_queries = {
    "health": [
        "healthcare ghana", "medical care ghana", "health care ghana",
        "quality health ghana", "poor health ghana"
    ],
    "transportation": [
        "road network ghana",
        "transportation ghana",
        "fuel price ghana",
    ],
    "education": [
        "education in ghana",
        "teaching learning in ghana",
        "ghana education system",
    ],
    "food": [
        "food price ghana", "food cost ghana", "meals per day system",
        "meal a day system", "diet pattern", "diet change"
    ],
}

In [116]:
# Get tweets
for sector, sector_phrases in sector_queries.items():
    # Pandas dataframe to store the data
    tweet_data_frame = pd.DataFrame(columns=[
        'tweet_id', 'tweet_text', 'sector', 'sentiment_score', 'neg_pos'
    ])
    records = []
    for phrase in sector_phrases:
        print(f"Getting tweets for '{phrase}'")

        query = "+".join(phrase.split())
        url = f"https://api.twitter.com/2/tweets/search/recent?max_results=100&expansions=geo.place_id&query={query} -is:retweet"
        res = requests.get(url, headers=headers)

        # Check if the response is valid
        if res.status_code != 200 or res.json().get("data") is None:
            print("Could not get tweets for phrase: {}".format(phrase))
            continue

        for tweet in res.json().get("data"):
            tweet_object = {
                "tweet_id": tweet.get("id"),
                "tweet_text": tweet.get("text"),
                "sector": sector
            }
            records.append(tweet_object)

        # Fetch next the pages of this query
        while res and res.json().get("meta") and res.json().get("meta").get(
                "next_token"):
            print("Fetching next page for phrase '{}'".format(phrase))

            next_token = res.json().get("meta").get("next_token")
            url += "&next_token={}".format(next_token)
            res = requests.get(url, headers=headers)

            # Check if the response is valid
            if res.status_code != 200 or res.json().get("data") is None:
                print("No more tweets for '{}'".format(phrase))
                continue

            for tweet in res.json()["data"]:
                tweet_object = {
                    "tweet_id": tweet.get("id"),
                    "tweet_text": tweet.get("text"),
                    "sector": sector
                }
                records.append(tweet_object)

    tweet_data_frame = pd.concat(
        [tweet_data_frame,
         pd.DataFrame.from_records(records)])

    # Save the sector data to a csv file
    tweet_data_frame.to_csv(sector + '.csv', index=False)

Getting tweets for 'healthcare ghana'
Getting tweets for 'medical care ghana'
Getting tweets for 'health care ghana'
Getting tweets for 'quality health ghana'
Getting tweets for 'poor health ghana'
Getting tweets for 'road network ghana'
Getting tweets for 'transportation ghana'
Getting tweets for 'fuel price ghana'
Getting tweets for 'education in ghana'
Fetching next page for phrase 'education in ghana'
Fetching next page for phrase 'education in ghana'
No more tweets for 'education in ghana'
Getting tweets for 'teaching learning in ghana'
Getting tweets for 'ghana education system'
Getting tweets for 'food price ghana'
Getting tweets for 'food cost ghana'
Getting tweets for 'meals per day system'
Getting tweets for 'meal a day system'
Getting tweets for 'diet pattern'
Getting tweets for 'diet change'
Fetching next page for phrase 'diet change'
Fetching next page for phrase 'diet change'
No more tweets for 'diet change'


## Merge all the sectors

In [120]:
df_exists = False
dfs = []
for file_name in sector_queries.keys():
    try:
        dfs += [pd.read_csv(file_name + ".csv")] 
        df_exists = True
    except FileNotFoundError:
        pass
        print(file_name + ".csv" + " not found.")
if df_exists:
    df = pd.concat(dfs)
    df.to_csv("all_sectors.csv",index=False)
else:
    print("No data to save.")

In [118]:
df.describe()

,tweet_id,sentiment_score,neg_pos,Unnamed: 0
count,5.169000e+03,0.0,0.0,1845.000000
mean,1.573094e+18,NaN,NaN,112.027100
std,6.537733e+14,NaN,NaN,78.548791
min,1.571454e+18,NaN,NaN,0.000000
25%,1.572563e+18,NaN,NaN,46.000000
50%,1.573208e+18,NaN,NaN,92.000000
75%,1.573719e+18,NaN,NaN,178.000000
max,1.574010e+18,NaN,NaN,270.000000


## Sentiment analysis